# **Teste técnico -- Base dos Dados**
### <font color='red'> *Análise de dados do Censo Agropecuário do Brasil de 1985 - 2017*
*by [Matheus Fellipe do Carmo Barros](https://linktr.ee/matheusfcbarros)*

---


### <font color=blue> Importando bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import basedosdados as bd
import matplotlib.pyplot as plt
import plotly
import plotly.express as px

### <font color=blue> 1. Carregando a base de dados

> Utilizei o pacote ``basedosdados`` para fazer a consulta SQL na BD+

Usei a base Censo Agropecuário (br_ibge_censo_agropecuario) para realizar as análises e realizei um cruzamento com a base 
(br_bd_diretorios_brasil) para poder ter também os nomes dos municípios. Ambas as bases estão disponíveis no site da [Base dos Dados](https://basedosdados.org/dataset?order_by=score)


> Query SQL, para download da base, em uma forma mais simples

In [ ]:
base_censo1 = bd.read_sql(
    '''
    SELECT censo.*, municipio.nome
        FROM basedosdados.br_ibge_censo_agropecuario.municipio AS censo 
        INNER JOIN basedosdados.br_bd_diretorios_brasil.municipio AS municipio
            ON censo.id_municipio = municipio.id_municipio
    ''',
    billing_project_id='teste1-360117'
)

> Query SQL, para download da base, usando subqueries

In [ ]:
base_censo2 = bd.read_sql(
    '''
    SELECT censo.*, nome_municipio.nome
        FROM basedosdados.br_ibge_censo_agropecuario.municipio AS censo 
        INNER JOIN (SELECT municipio.id_municipio, municipio.nome
                        FROM basedosdados.br_bd_diretorios_brasil.municipio AS municipio) AS nome_municipio
            ON censo.id_municipio = nome_municipio.id_municipio
    ''',
    billing_project_id='teste1-360117'
)

> Usando ``read_table``

In [ ]:
#Teste usando read_table
df = bd.read_table(dataset_id='br_ibge_censo_agropecuario',
table_id='municipio',
billing_project_id="teste1-360117")

Como forma de teste fiz o download da base de dados ``br_ibge_censo_agropecuario`` acrescida da informação dos nomes dos municípios e utlizando ``read_sql`` com uma query mais simples e ele foi realizado em 30s. Já o download da mesma base de dados utilizando subqueries foi realizado em 37s. O download da base ``br_ibge_censo_agropecuario`` , sem a informação dos nomes dos municípios , utilizando ``read_table`` gastou 49.8s.

### <font color=blue> 2. Informações da base de dados

In [ ]:
base_censo1

In [ ]:
#Tipos presente na base
base_censo1.dtypes

In [ ]:
base_censo1.info()

### <font color=blue> 3. Tratando a base de dados

In [ ]:
#convertendo id_municipio em tipo numerico para poder realizar uma melhor verificação de valores vazios
base_censo1["id_municipio"] = pd.to_numeric(base_censo1["id_municipio"], errors = "coerce")
base_censo1

In [ ]:
#Tipos presente na base após conversão da id_municipio
base_censo1.dtypes

In [ ]:
#Reordenando a base para uma melhor vizualização
col_nome = base_censo1["nome"]
base_censo1_ordenada = base_censo1
base_censo1_ordenada.insert(3,'nome_municipio',col_nome)


In [ ]:
#Base reordenada
base_censo1_ordenada

In [ ]:
#Verificando valores vazios para saber se posso eliminar alguma coluna
contar_Valores_NaN = base_censo1_ordenada.isnull().sum().sort_values(ascending = False)
contar_Valores_NaN 



In [ ]:
#Eliminando a coluna nome (Ja reordenei ela para outra posição) e eliminando a coluna (proporcao_despesa_salarios) ja que ela nao tem nenhum valor
base_censo1_ordenada= base_censo1_ordenada.drop(columns=["proporcao_despesa_salarios","nome"])
base_censo1_ordenada

        Resolvi preencher os valores vazios como 0 por entender que esses campos podem ser analisados 
        como informações que naquele respectivo ano não existiram naquela propriedade.

In [ ]:
#Preenchendo valores vazios
base_censo1_ordenada = base_censo1_ordenada.fillna(0)
base_censo1_ordenada


### <font color=blue> 4. Preparação dos dados para visualização

### Bovinos

In [ ]:
#Área utilizada por ano
total_area_pastagem = base_censo1_ordenada.groupby("ano").area_pastagem.sum()
total_area_pastagem = total_area_pastagem.reset_index()
total_area_pastagem

In [ ]:
#Cálculo da quantidade de bovinos por ano
total_quantidade_bovinos = base_censo1_ordenada.groupby("ano").quantidade_bovinos_total.sum()
total_quantidade_bovinos = total_quantidade_bovinos.reset_index()
total_quantidade_bovinos

In [ ]:
#Cálculo do valor da produção de bovinos por ano
valor_producao_total_animal = base_censo1_ordenada.groupby("ano").valor_producao_total_animal.sum()
valor_producao_total_animal = valor_producao_total_animal.reset_index()
valor_producao_total_animal



### Arroz

In [ ]:
#Área utilizada por ano
total_area_arroz = base_censo1_ordenada.groupby("ano").area_arroz.sum()
total_area_arroz = total_area_arroz.reset_index()
total_area_arroz


In [ ]:
#Cálculo da produção de arroz, em toneladas, por ano
total_producao_arroz = base_censo1_ordenada.groupby("ano").producao_total_arroz.sum()
total_producao_arroz = total_producao_arroz.reset_index()
total_producao_arroz

In [ ]:
#Cálculo do valor da produção de arroz por ano
valor_total_arroz = base_censo1_ordenada.groupby("ano").valor_total_arroz.sum()
valor_total_arroz = valor_total_arroz.reset_index()
valor_total_arroz

### Soja

In [ ]:
#Área utilizada no período
total_area_soja= base_censo1_ordenada.groupby("ano").area_soja.sum()
total_area_soja = total_area_soja.reset_index()
total_area_soja

In [ ]:
#Cálculo da produção de soja, em toneladas, no período 
total_producao_soja = base_censo1_ordenada.groupby("ano").producao_total_soja.sum()
total_producao_soja = total_producao_soja.reset_index()
total_producao_soja

In [ ]:
#Calculo do valor da produção de soja no período
valor_total_soja = base_censo1_ordenada.groupby("ano").valor_total_soja.sum()
valor_total_soja = valor_total_soja.reset_index()
valor_total_soja

> ``Dataframes auxiliares``

Criei esses dataframes auxiliares para agregar os valores calculados anteriormente e facilitar a visualização

In [ ]:
bovinos = pd.merge(total_area_pastagem, total_quantidade_bovinos, how= 'outer' )
bovinos = pd.merge(bovinos, valor_producao_total_animal, how= 'outer' )
bovinos

In [ ]:
arroz = pd.merge(total_area_arroz, total_producao_arroz, how= 'outer' )
arroz = pd.merge(arroz, valor_total_arroz, how= 'outer' )
arroz

In [ ]:
soja = pd.merge(total_area_soja, total_producao_soja, how= 'outer' )
soja = pd.merge(soja, valor_total_soja, how= 'outer' )
soja

### <font color=blue> 5. Gerando Gráficos

In [ ]:
graf_Bovinos = px.bar(bovinos, x = 'ano', y = 'quantidade_bovinos_total', color = 'area_pastagem')
plotly.offline.plot(graf_Bovinos)
graf_Bovinos.show()

In [ ]:
graf_arroz = px.bar(arroz, x = 'ano', y = 'producao_total_arroz', color = 'area_arroz')
plotly.offline.plot(graf_arroz)
graf_arroz.show()

In [ ]:
graf_Soja = px.bar(soja, x = 'ano', y = 'producao_total_soja', color = 'area_soja')
plotly.offline.plot(graf_Soja)
graf_Soja.show()